End to end floodplain mesh gen:
----------------------------------
```
Please use OCSMesh > version 1.6.4

Needs to prepare the following folders:
    path/
    |-- endtoend.py
    |-- inputs/
    |   |-- hgrid.gr3
    |   |-- ocean_mesh.2dm
    |   |-- total_river_polys.shp
    |   |-- dems/
    |       |-- gebco_file.tif  
    |-- outputs/
```
- **path:** your directory

- **hgrid.gr3:** STOFS3D mesh, used for defining the floodplain domain.
    Please download from: https://ccrm.vims.edu/yinglong/feiye/Public/OCSMesh/

- **ocean_mesh.2dm:** Ocean mesh, merged to the river+floodplain mesh at the end.
    Please download from: https://ccrm.vims.edu/yinglong/feiye/Public/OCSMesh/

- **total_river_polys.shp:** RiverMapper polygons, used to create the river mesh.
    Please download from: https://ccrm.vims.edu/yinglong/feiye/Public/OCSMesh/
    For details on how to create this file, please refer to: https://github.com/schism-dev/RiverMeshTools

- **gebco_file.tif:** should be placed inside dems/, should cover the entire model domain.
    Please download it from: https://download.gebco.net/
    Using the coordinates: -98.5 -51.0, 3, 53

- **outputs/:** where outputs (final mesh) will be saved.


In [3]:
import os
import time
from copy import deepcopy

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import (
                              Polygon, MultiPolygon, mapping
                             )
from shapely import intersection,difference

import ocsmesh

In [6]:
path = r"/Your/Path/Here/"

## Floodplain Domain: 

This step is only needed if you do not have a shapefile of the floodplain domain and wants it to match that of STOFS

In [ ]:
print("Begin Deliniating the Floodplain Domain")
start_time = time.time()

STOFS_mesh = ocsmesh.Mesh.open(path+"inputs/hgrid.gr3", crs=4326)
oc_mesh = ocsmesh.Mesh.open(path+"inputs/ocean_mesh.2dm", crs=4326)

poly_STOFS = ocsmesh.utils.get_mesh_polygons(STOFS_mesh.msh_t)
poly_oc = ocsmesh.utils.get_mesh_polygons(oc_mesh.msh_t)

fp_c = poly_STOFS.difference(poly_oc) #clipped floodplain
gdf = gpd.GeoDataFrame(geometry = gpd.GeoSeries(fp_c),crs=4326).dissolve().explode()
gdf.geometry = ocsmesh.utils.remove_holes(gdf.union_all()) #closing all holes in the polygons
gdf= gdf[gdf.geometry.area >= 1e-3] #removing slivers based on area

gdf['geometry'] = gdf.geometry.buffer(0.01)
gdf = gdf.dissolve().explode()
gdf.geometry = ocsmesh.utils.remove_holes(gdf.union_all())
gdf.to_file(path+"outputs/fp_domain.shp")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for Floodplain deliniation: {elapsed_time} seconds")

## River Mesh:

Triangulates the RiverMapper polygons and clips it for the area of interest (floodplain domain)

In [ ]:
print("Begin River Mesh Gen")
start_time = time.time()

rm_poly = gpd.read_file(path+"inputs/total_river_polys.shp")
river_tr = ocsmesh.utils.triangulate_rivermapper_poly(rm_poly)
river_tr = ocsmesh.utils.clip_mesh_by_shape(river_tr, gdf.union_all(), adjacent_layers=10)
ocsmesh.Mesh(river_tr).write(path+"outputs/river_tr.2dm", format='2dm', overwrite=True)
del rm_poly, river_tr

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for RiverMesh: {elapsed_time} seconds")

## Floodplain Mesh:

Floodplain mesh, hfun. allows customization. Please see the OCSMesh manual: https://repository.library.noaa.gov/view/noaa/33879

In [ ]:
print("Begin Floodplain Mesh Gen")
start_time = time.time()

dem_paths = [f"{path}inputs/dems/{dem}" for dem in os.listdir(path+"inputs/dems/")]
geom_rast_list = [ocsmesh.Raster(f) for f in dem_paths if f[-4:] == '.tif']
hfun_rast_list = [ocsmesh.Raster(f) for f in dem_paths if f[-4:] == '.tif']

#Mesh gen:
geom = ocsmesh.Geom(
    geom_rast_list,
    base_shape=gdf.union_all(),
    base_shape_crs=gdf.crs,
    # zmax=10
    )
hfun = ocsmesh.Hfun(
    hfun_rast_list,
    base_shape=gdf.union_all(),
    base_shape_crs=geom.crs,
    hmin=500, hmax=10000,
    method='fast')

hfun.add_constant_value(1200, lower_bound=-999990, upper_bound=-5)
hfun.add_constant_value(600, lower_bound=-5, upper_bound=99999)

driver = ocsmesh.JigsawDriver(geom, hfun, crs=4326)
fp_mesh = driver.run()

hfun_mesh = ocsmesh.mesh.EuclideanMesh2D(hfun.msh_t())
ocsmesh.utils.reproject(mesh=hfun_mesh.msh_t,dst_crs=4326)
fp_mesh = ocsmesh.utils.fix_small_el(fp_mesh, hfun_mesh, u_limit = 1e-7)
ocsmesh.utils.cleanup_isolates(fp_mesh)
ocsmesh.utils.cleanup_duplicates(fp_mesh)
ocsmesh.utils.put_id_tags(fp_mesh)

ocsmesh.Mesh(fp_mesh).write(path+"outputs/fp_mesh.2dm", format="2dm", overwrite=True)
del geom, hfun, geom_rast_list, hfun_rast_list, driver

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for FloodplainMesh: {elapsed_time} seconds")

## Merging Meshes:

This is a two step procedure.
First the river mesh is merged into the floodplain mesh, preserving the river mesh and carving out the floodplain mesh.
Then, the floodplain+river mesh is merged with the ocean mesh, preserving the ocean mesh and carving out the floodplain+river mesh.

In [ ]:
print("Begin River+Floodplain Mesh Merge")
start_time = time.time()

###Merge River into the Floodplain
fp_mesh = ocsmesh.Mesh.open(path+"outputs/fp_mesh.2dm", crs=4326)
river_mesh = ocsmesh.Mesh.open(path+"outputs/river_tr.2dm", crs=4326)

fp_r = ocsmesh.utils.merge_overlapping_meshes([fp_mesh.msh_t,river_mesh.msh_t])
fp_r = ocsmesh.utils.remesh_holes(fp_r, area_threshold_min = 0 , area_threshold_max = 0.002) #remove undesirable islands and slivers
ocsmesh.Mesh(fp_r).write(path+"outputs/fp_r.2dm", format='2dm', overwrite=True)

del fp_mesh, river_mesh
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for River+Floodplain Mesh Merge: {elapsed_time} seconds")


print("Begin RiverFloodplain+Ocean Mesh Merge")
start_time = time.time()
###Merge Floodplain+River with the Ocean
fp_r_o = ocsmesh.utils.merge_overlapping_meshes([fp_r, oc_mesh.msh_t])
fp_r_o = ocsmesh.utils.remesh_holes(fp_r_o, area_threshold_min = 0 , area_threshold_max = 1e-10) #remove possible slivers
ocsmesh.Mesh(fp_r_o).write(path+"outputs/fp_r_o.2dm", format='2dm', overwrite=True)
ocsmesh.Mesh(fp_r_o).write(path+"outputs/hgrid.ll", format='grd', overwrite=True)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for RiverFloodplain+Ocean Mesh Merge: {elapsed_time} seconds")